Import functions

In [1]:
import pandas as pd

In [18]:
df=pd.read_parquet('/mnt/research-live/user/yzhong/Matching_news_data/bloomberg_news_chinese_matching_2018_11_18.parquet')

In [ ]:
import pandas as pd

# Read in your original dataframe
df = pd.read_csv("your_data.csv")

# Define a function to check if a pair satisfies the conditions for a possible match
def is_possible_match(delay, cosine):
    if delay < 90 and cosine > 0.55:
        return True
    elif delay < 10 and cosine > 0.44:
        return True
    elif cosine > 0.85:
        return True
    else:
        return False

# Define a function to find the best match among the possible matches
def find_best_match(matches):
    if len(matches) == 1:
        return matches[0]
    else:
        # Sort matches by delay (smallest to largest)
        matches = sorted(matches, key=lambda x: x[0])
        # Choose the match with the smallest delay
        best_match = matches[0]
        # If there are ties, choose the match with the highest cosine similarity
        for match in matches:
            if match[0] == best_match[0] and match[1] > best_match[1]:
                best_match = match
        return best_match

# Create a new column to hold the match information
df["match"] = False

# Loop through the rows of the dataframe and look for matches
for i, row in df.iterrows():
    # Check each possible match
    possible_matches = []
    for j in range(1, 4):
        delay = row[f"delay{j}"]
        cosine = row[f"cos{j}"]
        if is_possible_match(delay, cosine):
            possible_matches.append((delay, cosine, f"Sim{j}"))
    # If there are no possible matches, move on to the next row
    if len(possible_matches) == 0:
        continue
    # Find the best match among the possible matches
    best_match = find_best_match(possible_matches)
    # Update the "match" column with the name of the matching column
    df.at[i, "match"] = best_match[2]

# Print the updated dataframe
print(df)


In [19]:
with open('/mnt/research-live/user/yzhong/match_2018_11_18.txt', 'w') as f:
    for row in df.itertuples():
        f.write(str(row.headline_org) + '\n')
        f.write(str(row.translation) + '\n')
        f.write(str(row.last_update) + '\n')
        f.write(f' Sim1: {row.cos1} {row.date1} {row.Sim1}' + '\n')
        f.write(f' Sim2: {row.cos2} {row.date2} {row.Sim2}' + '\n')
        f.write(f' Sim3: {row.cos3} {row.date3} {row.Sim3}' + '\n')
 

In [13]:
df_e=pd.read_parquet('/mnt/research-live/user/yzhong/English_news_data/2018/11/bloomberg_news_english_2018_11_18.parquet').sort_values('last_update')

In [14]:
with open('/mnt/research-live/user/yzhong/english_2018_11_18.txt', 'w') as f:
    for row in df_e.itertuples():
        f.write(str(row.headline) + '\n')
        f.write(str(row.last_update) + '\n')

In [ ]:
def plot_hist(df,col='delay'):
    for j in range(3):
        plt.hist(df[f'{col}{j+1}'], bins=50, alpha=0.5, label=f'sim{j+1}')

    # plt.hist(df['delay1'], bins=50, alpha=0.5, label='Date1')
    # plt.hist(df['delay2'], bins=50, alpha=0.5, label='Date2')
    # plt.hist(df['delay3'], bins=50, alpha=0.5, label='Date3')
    # Add axis labels and a legend
    if col=='delay':
        label='Delay (minutes)'
        
    if col == 'cos':
        label='Cosinus similarity'
    title=f'Distribution of the {label} of the Chinese news with English news'

    plt.xlabel(label)
    plt.ylabel('Frequency')
    plt.title(title)
    plt.legend()

    # Show the plot
    plt.show()
def plot_prob(df,col):
    hists=[None] * 3
    bin_edges=[None] * 3
    bin_centers=[None] * 3
    fig, ax = plt.subplots(figsize=(10, 5))
    for j in range(3):
        hists[j], bin_edges[j] = np.histogram(df[f'{col}{j+1}'], bins=50, density=True)
        bin_centers[j] = (bin_edges[j][:-1] + bin_edges[j][1:]) / 2
        ax.plot(bin_centers[j], hists[j], label=f'Sim{j+1}')
      
    # Add axis labels and a legend
    ax.set_xlabel(col)
    ax.set_ylabel('Probability density')
    ax.legend()

    # Show the plot
    plt.show()
def find_star (df):
    return ( df['headline_org'].str.contains(r'^\*[\u4e00-\u9fff0-9a-zA-Z].*'))
def get_TF(x, info_list):
    return any([True for i in info_list if i in str(x)])
def find_words(df, col_name='body', my_words=None):
    
    if my_words is None:
        my_words = ['原文标题','彭博自动新闻','原文標題']
        return (df[col_name].apply(lambda x: get_TF(x, my_words)))       

Read data

In [ ]:
import pandas as pd
import os

# Get a list of all the parquet files in the directory
parquet_files = [os.path.join('/mnt/research-live/user/yzhong/Matching_news_data', f) for f in os.listdir('/mnt/research-live/user/yzhong/Matching_news_data') if f.endswith('.parquet')]
# Concatenate the parquet files into a single dataframe
df = pd.concat([pd.read_parquet(f) for f in parquet_files])
# Delete the "id" column
df = df.drop('id', axis=1)
# Reset the index
df = df.reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
# Convert columns to datetime type
df['last_update'] = pd.to_datetime(df['last_update'])
df['date1'] = pd.to_datetime(df['date1'])
df['date2'] = pd.to_datetime(df['date2'])
df['date3'] = pd.to_datetime(df['date3'])
# Calculate the delay in minutes between last_update and date1, date2, and date3
df['delay1'] = (df['last_update'] - df['date1']).dt.total_seconds() / 60
df['delay2'] = (df['last_update'] - df['date2']).dt.total_seconds() / 60
df['delay3'] = (df['last_update'] - df['date3']).dt.total_seconds() / 60

Compare with patterns 

In [ ]:
#df=df_sim.copy(deep=True)
mask1=find_star(df)
mask2=find_words(df)
df['contains_star']=mask1
df['contains_words']=mask2
df['Only_Ch']= (~mask1) & (~mask2)